In [1]:
import pandas as pd
import os
os.getcwd()

'/home/niek/Elvina/Thesis/repository/Fact_Checking_model/explore'

In [2]:
df = pd.read_csv('../data/raw/MovieSummaries/character.metadata.tsv', delimiter='\t', header=None)
column_names = ['Wikipedia ID','Freebase movie ID','Movie release date', 'Character name', 'Actor dob', 'Gender', 'Height','Ethnicity','Actor','Age', 'Freebase character/actor map ID','Freebase character ID', 'Freebase actor ID']

df.columns = column_names
#df = df.rename_axis("Wikipedia ID").reset_index()




   Wikipedia ID Freebase movie ID Movie release date  \
0        975900         /m/03vyhn         2001-08-24   
1        975900         /m/03vyhn         2001-08-24   
2        975900         /m/03vyhn         2001-08-24   
3        975900         /m/03vyhn         2001-08-24   
4        975900         /m/03vyhn         2001-08-24   
5        975900         /m/03vyhn         2001-08-24   
6        975900         /m/03vyhn         2001-08-24   
7        975900         /m/03vyhn         2001-08-24   
8        975900         /m/03vyhn         2001-08-24   
9        975900         /m/03vyhn         2001-08-24   

               Character name   Actor dob Gender  Height   Ethnicity  \
0                    Akooshay  1958-08-26      F   1.620         NaN   
1  Lieutenant Melanie Ballard  1974-08-15      F   1.780  /m/044038p   
2         Desolation Williams  1969-06-15      M   1.727     /m/0x67   
3          Sgt Jericho Butler  1967-09-12      M   1.750         NaN   
4             Bashira K

In [ ]:
df.drop(['Movie release date', 'Character name', 'Actor dob', 'Gender', 'Height','Ethnicity','Age', 'Freebase character/actor map ID','Freebase character ID', 'Freebase actor ID'], axis=1, inplace=True)


In [8]:

actors_df = df
print(actors_df)

        Wikipedia ID Freebase movie ID               Actor
0             975900         /m/03vyhn      Wanda De Jesus
1             975900         /m/03vyhn  Natasha Henstridge
2             975900         /m/03vyhn            Ice Cube
3             975900         /m/03vyhn       Jason Statham
4             975900         /m/03vyhn         Clea DuVall
...              ...               ...                 ...
450664        913762         /m/03pcrp  Dorothy Elias-Fahn
450665        913762         /m/03pcrp       Jonathan Fahn
450666      28308153        /m/0cp05t9      David Hemmings
450667      28308153        /m/0cp05t9    Roberta Paterson
450668      28308153        /m/0cp05t9         John Rogers

[450669 rows x 3 columns]


## Metadata


In [73]:
data = pd.read_csv('../data/raw/MovieSummaries/movie.metadata.tsv', delimiter='\t', header=None)

In [74]:
column_names = ['Wikipedia ID','Freebase movie ID','Movie name', 'Year', 'Movie box office revenue', 'Movie runtime', 'Movie languages', 'Movie countries', 'Genre']
data.columns = column_names
data.drop(['Movie box office revenue', 'Movie runtime', 'Movie languages', 'Movie countries'], axis=1, inplace=True)

data['Genre'] = data['Genre'].apply(lambda x: eval(x).values())
print(data.head(6))

   Wikipedia ID Freebase movie ID  \
0        975900         /m/03vyhn   
1       3196793         /m/08yl5d   
2      28463795        /m/0crgdbh   
3       9363483        /m/0285_cd   
4        261236         /m/01mrr1   
5      13696889        /m/03cfc81   

                                          Movie name        Year  \
0                                     Ghosts of Mars  2001-08-24   
1  Getting Away with Murder: The JonBenét Ramsey ...  2000-02-16   
2                                        Brun bitter        1988   
3                                   White Of The Eye        1987   
4                                  A Woman in Flames        1983   
5                                      The Gangsters  1913-05-29   

                                               Genre  
0  (Thriller, Science Fiction, Horror, Adventure,...  
1   (Mystery, Biographical film, Drama, Crime Drama)  
2                             (Crime Fiction, Drama)  
3  (Thriller, Erotic thriller, Psychologica

In [77]:
data['Year'] = data['Year'].astype(str).str[:4]
grouped_years = data.groupby('Year').size()
grouped_years

Year
1010       1
1888       1
1889       1
1890       2
1891       3
        ... 
2013     164
2014      17
2015       4
2016       3
nan     6902
Length: 131, dtype: int64

In [80]:
data.loc[data['Year'] == '1010', 'Year'] = '2010'

In [81]:
grouped_years = data.groupby('Year').size()
print(grouped_years)


Year
1888       1
1889       1
1890       2
1891       3
1892       3
        ... 
2013     164
2014      17
2015       4
2016       3
nan     6902
Length: 130, dtype: int64


In [62]:

data['Year'] = data.loc[data['Year'] == 1010, 'Year'] = 2010

Year
2010    81741
dtype: int64
